### Using the TensorDataset class
In practice, loading your data into a PyTorch dataset will be one of the first steps you take in order to create and train a neural network with PyTorch.

The TensorDataset class is very helpful when your dataset can be loaded directly as a NumPy array. Recall that TensorDataset() can take one or more NumPy arrays as input.

In [32]:
import torch 
import numpy as np
import pandas as pd
import torch.nn as nn 
import torch.optim as optim 
import torchmetrics
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
np_features = np.array(np.random.rand(12, 8))
np_target = np.array(np.random.rand(12, 1))

In [7]:
torch_features = torch.from_numpy(np_features)
torch_target = torch.from_numpy(np_target)

In [8]:
dataset = TensorDataset(torch_features, torch_target)

In [9]:
print(dataset[-1])

(tensor([0.2800, 0.5746, 0.7542, 0.5703, 0.1399, 0.4265, 0.8927, 0.3660],
       dtype=torch.float64), tensor([0.3140], dtype=torch.float64))


### From data loading to running a forward pass
In this exercise, you'll create a PyTorch DataLoader from a pandas DataFrame and call a model on this dataset. Specifically, you'll run a forward pass on a neural network. You'll continue working with fully connected neural networks, as you have done so far.

You'll begin by subsetting a loaded DataFrame called dataframe, converting features and targets NumPy arrays, and converting to PyTorch tensors in order to create a PyTorch dataset.

This dataset can be loaded into a PyTorch DataLoader, batched, shuffled, and used to run a forward pass on a custom fully connected neural network.

In [16]:
dataframe = pd.read_csv('../../2. IntermediateDeepLearningWithPytorch/data/water_potability/water_train.csv')


In [17]:
dataframe.columns

Index(['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability'],
      dtype='object')

In [19]:
features = torch.tensor(dataframe[['ph', 'Sulfate', 'Conductivity', 'Organic_carbon']].to_numpy()).float()
target = torch.tensor(dataframe['Potability'].to_numpy()).float()

In [22]:

dataset = TensorDataset(features, target)

In [23]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [24]:
x, y = next(iter(dataloader))

In [25]:
x.shape

torch.Size([2, 4])

In [26]:
model = nn.Sequential(
    nn.Linear(x.shape[1], 4 ), 
    nn.Linear(4, 1)
    )

output = model(features)

In [27]:
print(output)

tensor([[ 0.0233],
        [ 0.1342],
        [-0.0520],
        ...,
        [ 0.1828],
        [ 0.1050],
        [ 0.1207]], grad_fn=<AddmmBackward0>)


### Writing the evaluation loop
In this exercise, you will practice writing the evaluation loop. Recall that the evaluation loop is similar to the training loop, except that you will not perform the gradient calculation and the optimizer step.

In [28]:
criterion = nn.CrossEntropyLoss()

In [29]:
validationloader = DataLoader(TensorDataset(features, target), shuffle=True, batch_size=2)

In [31]:
model.eval()

validation_loss = 0.0

with torch.no_grad():
    for data in validationloader:

        outputs = model(data[0])

        loss = criterion(outputs, data[1])

        validation_loss += loss.item()

validation_loss_epoch = validation_loss / len(validationloader)
print(validation_loss_epoch)


IndexError: Target 1 is out of bounds.

### Calculating accuracy using torchmetrics
In addition to the losses, you should also be keeping track of the accuracy during training. By doing so, you will be able to select the epoch when the model performed the best.

In this exercise, you will practice using the torchmetrics package to calculate the accuracy. You will be using a sample of the facemask dataset. This dataset contains three different classes. The plot_errors function will display samples where the model predictions do not match the ground truth. Performing such error analysis will help you understand your model failure modes.

In [33]:
from mpl_toolkits.axes_grid1 import ImageGrid

In [34]:
def plot_errors(model, dataloader): 
    # find mismatches
    mismatches = []
    for data in dataloader:
        if len(mismatches) > 8:
            break
        features, labels = data
        outputs = model(features)
        gt = labels.argmax(-1)
        pred = outputs.argmax(-1)
        for f, g, p in zip(features, gt, pred):
            if g != p:
                mismatches.append((f, g, p))
    
    
    fig = plt.figure(figsize=(8, 8))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(2, 4),  # creates 2x2 grid of axes
                     axes_pad=0.5,  # pad between axes in inch.
                     )
    mapping = {0: 'No mask', 1: 'Mask', 2: 'Incorrect'}
    for idx, ax in enumerate(grid):
        ax.imshow(mismatches[idx][0].permute(1, 2, 0))
        ax.set_title(f'GT: {mapping[mismatches[idx][1].item()]} \n PRED: {mapping[mismatches[idx][2].item()]}')
        ax.axis('off')
    plt.show()

In [35]:
metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 3)

for data in dataloader:

    features, target = data

    outputs = model(features)

    acc = metric(outputs.softmax(dim=-1), target.argmax(dim=-1)) #Calculate accuracy over the batch

# Calculate accuracy over the whole epoch
acc = metric.compute()

# Reset the metric for the next epoch (training or validation)
metric.reset()
plot_errors(model, dataloader)

ValueError: Either `preds` and `target` both should have the (same) shape (N, ...), or `target` should be (N, ...) and `preds` should be (N, C, ...).

### Experimenting with dropout
The dropout layer randomly zeroes out elements of the input tensor. Doing so helps fight overfitting. In this exercise, you'll create a small neural network with at least two linear layers, two dropout layers, and two activation functions.

The torch.nn package has already been imported as nn. An input_tensor of dimensions  has been created for you.

<span style="color:yellow">Note: Always the dropout should be after the activation function</span>.

In [37]:
input_tensor = torch.tensor([[0.1819, 0.1650, 0.1538, 0.2755, 0.2245, 0.1901, 0.3755, 0.2909, 0.2406,
        0.4506, 0.3518, 0.2998, 0.4987, 0.3827, 0.3141, 0.5221, 0.3999, 0.3266,
        0.5346, 0.4121, 0.3374, 0.5354, 0.4121, 0.3435, 0.5176, 0.3972, 0.3266,
        0.5366, 0.4158, 0.3460, 0.5698, 0.4495, 0.3836, 0.5887, 0.4712, 0.4198,
        0.6100, 0.5031, 0.4563, 0.6262, 0.5266, 0.4790, 0.6344, 0.5243, 0.4809,
        0.6329, 0.5253, 0.4833, 0.6328, 0.5291]])

In [38]:
# Using the same model, set the dropout probability to 0.8
model = nn.Sequential(
                    nn.Linear(input_tensor.shape[1], 16),
                    nn.ReLU(), 
                    nn.Dropout(p=0.8)
)
model(input_tensor)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2597,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4482, 0.0000]],
       grad_fn=<MulBackward0>)

### Implementing random search
Hyperparameter search is a computationally costly approach to experiment with different hyperparameter values. However, it can lead to performance improvements. In this exercise, you will implement a random search algorithm.

You will randomly sample 10 values of the learning rate and momentum from the uniform distribution. To do so, you will use the np.random.uniform() function.

In [39]:
values = []
for idx in range(10):
    # Randomly sample a learning rate factor between 2 and 4
    factor = np.random.uniform(2,4)
    lr = 10 ** -factor
    
    # Randomly select a momentum between 0.85 and 0.99
    momentum = np.random.uniform(0.85,0.99)
    
    values.append((lr, momentum))

In [40]:
values

[(0.0030157987987523656, 0.9583397995951717),
 (0.0038869594582657146, 0.9168495738540724),
 (0.002926113182659199, 0.9793419166434019),
 (0.0033302488708845243, 0.8524607277404614),
 (0.0006658977813435177, 0.9452997826928229),
 (0.002781808787365453, 0.8612637949632053),
 (0.000881704088360836, 0.9186851494286118),
 (0.006514858977718607, 0.946840026125843),
 (0.003072459619133891, 0.9877799664677912),
 (0.0018477116653312993, 0.977173786195438)]